<a href="https://colab.research.google.com/github/Koush98/Python/blob/main/Trading_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import pandas as pd
import numpy as np
import random

# Example implementation of getTickerPrice function
def getTickerPrice(ticker: str, date: pd.Timestamp) -> float:
    return random.uniform(1, 100)

def trade_performance_metrics(trades: pd.DataFrame) -> pd.Series:
    if trades.empty:
        return pd.Series(dtype='float64')

    trades['Size'] = trades['Size'].fillna(1)

    # Ensuring correct data types
    trades['Date'] = pd.to_datetime(trades['Date'])
    trades['Side'] = trades['Side'].astype(str)
    trades['Symbol'] = trades['Symbol'].astype(str)

    # Calculating PnL for each trade
    trades['Current Price'] = trades.apply(lambda row: getTickerPrice(row['Symbol'], row['Date']), axis=1)
    trades['PnL'] = trades.apply(lambda row: (row['Current Price'] - row['Price']) * row['Size']
                                 if row['Side'] == 'buy' else (row['Price'] - row['Current Price']) * row['Size'], axis=1)

    # Calculate metrics
    total_pnl = trades['PnL'].sum()
    win_rate = (trades['PnL'] > 0).mean()
    avg_win = trades[trades['PnL'] > 0]['PnL'].mean()
    avg_loss = trades[trades['PnL'] <= 0]['PnL'].mean()
    max_drawdown = (trades['PnL'].cumsum().expanding().max() - trades['PnL'].cumsum()).max()
    daily_returns = trades['PnL'].cumsum().pct_change().dropna()
    sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252)
    sortino_ratio = daily_returns.mean() / daily_returns[daily_returns < 0].std() * np.sqrt(252)
    profit_factor = trades[trades['PnL'] > 0]['PnL'].sum() / -trades[trades['PnL'] <= 0]['PnL'].sum()
    trade_duration = (trades['Date'].max() - trades['Date'].min()).days / len(trades)
    roi = total_pnl / trades['Price'].sum()

    metrics = {
        'Total PnL': total_pnl,
        'Win Rate': win_rate,
        'Average Win': avg_win,
        'Average Loss': avg_loss,
        'Max Drawdown': max_drawdown,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Profit Factor': profit_factor,
        'Trade Duration': trade_duration,
        'ROI': roi
    }

    return pd.Series(metrics)

In [96]:

# Loading the data
file_path = '/content/testData.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Ensuring the Date column is in datetime format
df['Date'] = pd.to_datetime(df['transactionDate'])

# Rename columns to match required format
df.rename(columns={
    'Date': 'Date',
    'ticker': 'Symbol',
    'type': 'Side',
    'amount': 'Price'
}, inplace=True)

df.drop(columns=['option_symbol', 'capitalGainsOver200USD','district','representative','assetDescription','owner','disclosureDate','disclosureYear','transactionDate'], inplace=True)


# Ensuring Size column is filled with 1 if missing
if 'Size' not in df.columns:
    df['Size'] = 1
else:
    df['Size'].fillna(1, inplace=True)

In [89]:
df

,Symbol,Side,Price,Date,Size
0,AAPL,Purchase,"$100,001 - $250,000",2023-05-20,1
1,GOOGL,Sale (Full),"$50,001 - $100,000",2023-04-10,1
2,AMZN,Purchase,"$100,001 - $250,000",2022-12-06,1
3,AMZN,Purchase,"$100,001 - $250,000",2022-04-14,1
4,MSFT,Purchase,"$1,001 - $15,000",2022-03-11,1
...,...,...,...,...,...
97,AAPL,Sale (Full),"$1,001 - $15,000",2023-07-26,1
98,TSLA,Sale (Full),"$50,001 - $100,000",2023-05-03,1
99,GOOGL,Sale (Partial),"$1,001 - $15,000",2023-09-16,1
100,AAPL,Sale (Full),"$100,001 - $250,000",2022-06-25,1


In [90]:
import pandas as pd
import re

def convert_price_range(price_range):
    # Removing dollar signs and commas
    price_range = price_range.replace('$', '').replace(',', '')

    # Splitting the range into two parts
    prices = re.split(r'[-–]', price_range.strip())

    # Convert to float and calculate the average if it's a range
    if len(prices) == 2:
        return (float(prices[0]) + float(prices[1])) / 2
    elif len(prices) == 1:
        return float(prices[0])
    else:
        raise ValueError("Invalid price range format")



# Apply the conversion function to the 'amount' column
df['Price'] = df['Price'].apply(convert_price_range)

# Print the updated DataFrame

# Mapping dictionary
mapping = {
    'Purchase': 'buy',
    'Sale (Partial)': 'sale',
    'Sale (Full)': 'sale'
}

# Replace the values
df['Side'] = df['Side'].replace(mapping)

df

,Symbol,Side,Price,Date,Size
0,AAPL,buy,175000.5,2023-05-20,1
1,GOOGL,sale,75000.5,2023-04-10,1
2,AMZN,buy,175000.5,2022-12-06,1
3,AMZN,buy,175000.5,2022-04-14,1
4,MSFT,buy,8000.5,2022-03-11,1
...,...,...,...,...,...
97,AAPL,sale,8000.5,2023-07-26,1
98,TSLA,sale,75000.5,2023-05-03,1
99,GOOGL,sale,8000.5,2023-09-16,1
100,AAPL,sale,175000.5,2022-06-25,1


In [95]:
# Example usage:
trades = df
metrics = trade_performance_metrics(trades)
print (metrics)

Total PnL         4.277870e+06
Win Rate          6.078431e-01
Average Win       1.399120e+05
Average Loss     -1.099169e+05
Max Drawdown      6.328225e+05
Sharpe Ratio      2.944981e+00
Sortino Ratio     6.599336e+00
Profit Factor     1.972978e+00
Trade Duration    7.058824e+00
ROI               3.271530e-01
dtype: float64
